## 特征工程

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.stats import chi2
import statsmodels.discrete.discrete_model as dm_

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

sys.path.insert(0, os.path.abspath('../'))

%matplotlib inline

from lec1.preprocessor import *

import toad

In [2]:
data_path = f"{os.path.abspath('.')}/home-credit-default-risk/"
application_train_orig, bureau_orig, bureau_balance_orig, credit_card_balance_orig, \
installment_payments_orig, pos_cash_balance_orig, previous_application_orig = load_data(data_path)

# Application数据

Value 365243 denotes infinity in DAYS variables in the datasets, therefore you can consider them NA values. Also XNA/XAP denote NA values.(https://www.kaggle.com/c/home-credit-default-risk/discussion/57247)

In [3]:
application_train = application_train_orig.copy()

In [4]:
application_train.loc[application_train['DAYS_EMPLOYED'] > 0]['DAYS_EMPLOYED'].unique()

array([365243], dtype=int64)

In [5]:
application_train['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

In [6]:
print('Categorical featrue values before label encoding', application_train['NAME_CONTRACT_TYPE'].unique())
application_train = process_cat_feature(application_train)
print('Categorical featrue values after label encoding',application_train['NAME_CONTRACT_TYPE'].unique())

Categorical featrue values before label encoding ['Cash loans' 'Revolving loans']
3 variable are label encoded
Categorical featrue values after label encoding [0 1]


In [7]:
print('Categorical featrue values before one-hot encoding: \n', application_train['CODE_GENDER'].value_counts())
application_train['CODE_GENDER'].replace('XNA',np.nan, inplace=True)
print('Categorical featrue values after converting na value: \n', application_train['CODE_GENDER'].value_counts())
application_train = one_hot_encoder(application_train)
print('Categorical featrue values after one-hot encoding: \n')
for x in [y for y in application_train.columns if 'CODE_GENDER' in y]:
    print(f'Column {x}')
    print(application_train[x].value_counts())
#     application_train[[x for x in application_train.columns if 'CODE_GENDER' in x]].info()

Categorical featrue values before one-hot encoding: 
 F      202448
M      105059
XNA         4
Name: CODE_GENDER, dtype: int64
Categorical featrue values after converting na value: 
 F    202448
M    105059
Name: CODE_GENDER, dtype: int64
The shape of dataset after One hot encoding: (307511, 255)
Categorical featrue values after one-hot encoding: 

Column CODE_GENDER_F
1    202448
0    105063
Name: CODE_GENDER_F, dtype: int64
Column CODE_GENDER_M
0    202452
1    105059
Name: CODE_GENDER_M, dtype: int64
Column CODE_GENDER_nan
0    307507
1         4
Name: CODE_GENDER_nan, dtype: int64


In [8]:
application_train_cleaned = application_cleaning(application_train_orig)
application_train_preprocessed = process_cat_feature(application_train_cleaned)
application_train_preprocessed = one_hot_encoder(application_train_preprocessed)
application_train_preprocessed.to_csv(data_path + 'application_train_processed.csv', index=False)

3 variable are label encoded
The shape of dataset after One hot encoding: (307511, 253)


In [9]:
print('top 10 features with high iv')
display(toad.quality(application_train_preprocessed.drop('SK_ID_CURR',axis=1),'TARGET',iv_only=True)[:10])
print('last 10 features with low iv')
display(toad.quality(application_train_preprocessed.drop('SK_ID_CURR',axis=1),'TARGET',iv_only=True)[-10:])


top 10 features with high iv


,iv,gini,entropy,unique
EXT_SOURCE_3,0.332758,NaN,NaN,815.0
EXT_SOURCE_2,0.321745,NaN,NaN,119832.0
EXT_SOURCE_1,0.154901,NaN,NaN,114585.0
DAYS_EMPLOYED,0.115184,NaN,NaN,12574.0
AMT_GOODS_PRICE,0.101163,NaN,NaN,1003.0
DAYS_BIRTH,0.090051,NaN,NaN,17460.0
AMT_CREDIT,0.068424,NaN,NaN,5603.0
DAYS_LAST_PHONE_CHANGE,0.053347,NaN,NaN,3773.0
REGION_RATING_CLIENT_W_CITY,0.051201,NaN,NaN,3.0
NAME_EDUCATION_TYPE_Higher education,0.049214,NaN,NaN,2.0


last 10 features with low iv


,iv,gini,entropy,unique
NAME_HOUSING_TYPE_Co-op apartment,1.325325e-06,NaN,NaN,2.0
ORGANIZATION_TYPE_Legal Services,7.592236e-07,NaN,NaN,2.0
NAME_INCOME_TYPE_Businessman,6.386774e-07,NaN,NaN,2.0
FLAG_DOCUMENT_20,6.161632e-07,NaN,NaN,2.0
ORGANIZATION_TYPE_Advertising,1.850903e-07,NaN,NaN,2.0
ORGANIZATION_TYPE_Industry: type 7,1.194622e-07,NaN,NaN,2.0
NAME_EDUCATION_TYPE_nan,0.000000e+00,NaN,NaN,1.0
NAME_INCOME_TYPE_nan,0.000000e+00,NaN,NaN,1.0
NAME_HOUSING_TYPE_nan,0.000000e+00,NaN,NaN,1.0
WEEKDAY_APPR_PROCESS_START_nan,0.000000e+00,NaN,NaN,1.0


# 其他数据

In [10]:
bureau = bureau_orig.copy()
bureau_balance = bureau_balance_orig.copy()
credit_card_balance = credit_card_balance_orig.copy()
installment_payments = installment_payments_orig.copy()
pos_cash_balance = pos_cash_balance_orig.copy()
previous_application = previous_application_orig.copy()

In [ ]:
bureau_cleaned = bureau_cleaning(bureau)
bureau_preprocessed = process_cat_feature(bureau_cleaned)
bureau_preprocessed = one_hot_encoder(bureau_preprocessed)
bureau_preprocessed.to_csv(data_path + 'bureau_preprocessed.csv', index=False)

D:\fal风险控制模型建模\资料\「风控模型」学习资料（week1）\fal-model-code\lec1\preprocessor.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DAYS_CREDIT_ENDDATE'][df['DAYS_CREDIT_ENDDATE'] < -40000] = np.nan
D:\fal风险控制模型建模\资料\「风控模型」学习资料（week1）\fal-model-code\lec1\preprocessor.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DAYS_CREDIT_UPDATE'][df['DAYS_CREDIT_UPDATE'] < -40000] = np.nan
D:\fal风险控制模型建模\资料\「风控模型」学习资料（week1）\fal-model-code\lec1\preprocessor.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

0 variable are label encoded
The shape of dataset after One hot encoding: (1716428, 40)


In [ ]:
bureau_balance_cleaned = bureau_balance_cleaning(bureau_balance)
bureau_balance_preprocessed = process_cat_feature(bureau_balance_cleaned)
bureau_balance_preprocessed = one_hot_encoder(bureau_balance_preprocessed)
bureau_balance_preprocessed.to_csv(data_path + 'bureau_balance_preprocessed.csv', index=False)

In [13]:
credit_card_balance_cleaned = credit_card_cleaning(credit_card_balance)
credit_card_balance_preprocessed = process_cat_feature(credit_card_balance_cleaned)
credit_card_balance_preprocessed = one_hot_encoder(credit_card_balance_preprocessed)
credit_card_balance_preprocessed.to_csv(data_path + 'credit_card_balance_preprocessed.csv', index=False)

/Users/chenlong.jia/Documents/project/home_credit/services/jia_home_credit/preprocessor.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AMT_DRAWINGS_ATM_CURRENT'][df['AMT_DRAWINGS_ATM_CURRENT'] < 0] = np.nan
/Users/chenlong.jia/Documents/project/home_credit/services/jia_home_credit/preprocessor.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AMT_DRAWINGS_CURRENT'][df['AMT_DRAWINGS_CURRENT'] < 0] = np.nan


0 variable are label encoded
The shape of dataset after One hot encoding: (3840312, 30)


In [14]:
installment_payments_cleaned = installment_payment_cleaning(installment_payments)
installment_payments_preprocessed = process_cat_feature(installment_payments_cleaned)
installment_payments_preprocessed = one_hot_encoder(installment_payments_preprocessed)
installment_payments_preprocessed.to_csv(data_path + 'installment_payments_preprocessed.csv', index=False)

0 variable are label encoded
The shape of dataset after One hot encoding: (13605401, 8)


In [15]:
pos_cash_balance_cleaned = pos_cash_cleaning(pos_cash_balance)
pos_cash_balance_preprocessed = process_cat_feature(pos_cash_balance_cleaned)
pos_cash_balance_preprocessed = one_hot_encoder(pos_cash_balance_preprocessed)
pos_cash_balance_preprocessed.to_csv(data_path + 'pos_cash_balance_preprocessed.csv', index=False)

0 variable are label encoded
The shape of dataset after One hot encoding: (10001358, 17)


In [16]:
previous_application_cleaned = prev_application_cleaning(previous_application)
previous_application_preprocessed = process_cat_feature(previous_application_cleaned)
previous_application_preprocessed = one_hot_encoder(previous_application_preprocessed)
previous_application_preprocessed.to_csv(data_path + 'previous_application_preprocessed.csv', index=False)

1 variable are label encoded
The shape of dataset after One hot encoding: (1670214, 178)


# categorical feature will be encoded
